In [30]:
import requests
import folium
import collections
from tinydb import TinyDB, Query
import pandas as pd

In [2]:
pip install tinydb

Note: you may need to restart the kernel to use updated packages.


In [31]:
# kakao api를 사용하기 위한 key값과 주소 설정
app_key = 'KakaoAK d659c09f558076c5e381a10bbe9a1d48'
url = 'https://dapi.kakao.com/v2/local/search/keyword.json'
params = {
    'query': '제로웨이스트',
    'page': 1,
    'rect': '',
}
headers  = {
    'Authorization': app_key
}
resp = requests.get(url, params=params, headers=headers)

In [32]:
def get_store_list(start_x,start_y,end_x,end_y):
    # 지도상 경계선에 들어가는 값을 줄이기 위한 offset값
    offset = 0.01
    #print(start_x,start_y,end_x,end_y)
    cnt = 1
    # 데이터가 담길 리스트
    resp_list = []
    while True:
        # 사각형 증가값 설정
        jump_x = 0.5
        jump_y = 0.5
        # 카카오 api 사용을 위한 headers와 params 설정
        params = {
            'query': '제로웨이스트',
            'page': cnt,
            'rect': f'{start_x-offset},{start_y-offset},{end_x+offset},{end_y+offset}'
        }
        headers  = {
            'Authorization': app_key
        }
        # requests.get으로 값 받아오기
        resp = requests.get(url, params=params, headers=headers)

        # 찾은 지점의 갯수 확인
        search_count = resp.json()['meta']['total_count']
        print(search_count)

        # 45개 이상이면 해당 지점 4분할 해서 지점 재확인
        if search_count > 45:
            print('big data dividing...')
            dividing_x = (start_x + end_x) / 2
            dividing_y = (start_y + end_y) / 2
            # 1
            resp_list.extend(get_store_list(start_x,start_y,dividing_x,dividing_y))
            # 2 x만 이동
            resp_list.extend(get_store_list(dividing_x,start_y,end_x,dividing_y))
            # 3 y만 이동
            resp_list.extend(get_store_list(start_x,dividing_y,dividing_x,end_y))
            # 4 x,y 전부 이동
            resp_list.extend(get_store_list(dividing_x,dividing_y,end_x,end_y))
            return resp_list
        # 1페이지에 15개의 데이터 출력되므로 1페이지면 resp_list에 저장
        else:
            if resp.json()['meta']['is_end']:
                resp_list.extend(resp.json()['documents'])
                return resp_list
            # 아니면 다음 페이지로 넘어가서 데이터 저장
            else:
                print('paging...')
                cnt += 1
                resp_list.extend(resp.json()['documents'])

# 위도, 경도 찾아오는 함수          
def getLatLng(addr):
    app_key = 'KakaoAKd659c09f558076c5e381a10bbe9a1d48'
    url = 'https://dapi.kakao.com/v2/local/search/address.json'
    params = {
        'query': addr,
        'page': '',
        'AddressSize': ''
    }
    headers  = {
        'Authorization': app_key
    }
    resp = requests.get(url, params=params, headers=headers)

    # 불러온 데이터가 정상이면 위도, 경도 리턴
    if resp.json()['documents'] != []:
        json = resp.json()['documents'][0]
        return (json['x'], json['y'])

    # 주소에 문제가 있는 경우 맨 뒤를 제거하며 주소로 재검색
    else: 
        fixed_addr = addr.replace(addr.split(' ')[-1], '').strip()
        if fixed_addr == '':
            print('불러오지 못했습니다 : ', addr)
            return (0, 0)
        else:
            return getLatLng(fixed_addr)

# 지도의 중간값 얻는 함수
def get_avg_point(x_list, y_list):
    x = sum(x_list) / len(x_list)
    y = sum(y_list) / len(y_list)
    return x, y

# 지도 그리는 함수
def draw_map(addr):
    map_noman = folium.Map(location=get_avg_point(addr_point_list),zoom_start=7)
    return map_noman

# 지도에 마커찍는 함수
def draw_marker(x, y, tel):
    marker = folium.Marker((x, y), popup=tel)
    icon=folium.Icon(color='blue',icon='glyphicon glyphicon-cutlery')
    icon.add_to(marker)
    return marker

In [33]:
app_key = 'KakaoAK d659c09f558076c5e381a10bbe9a1d48'
url = 'https://dapi.kakao.com/v2/local/search/keyword.json'

# 시작 x 좌표 및 증가값
start_x = 126 # ~ 129.7, 3.4
jump_x = 0.5
jump_y = 0.5

# 최종 데이터가 담길 리스트
zerowaste_list = []

# 지도를 사각형으로 나누면서 데이터 받아옴
for i in range(1,10):
    end_x = start_x + jump_x
    start_y = 33 # ~ 38.3, 4.9
    for j in range(1,13):
        end_y = start_y + jump_y
        zerowaste_list_one = get_store_list(start_x,start_y,end_x,end_y)
        zerowaste_list.extend(zerowaste_list_one)
        start_y = end_y
    start_x = end_x
#zerowaste

6
0
0
1
0
0
0
0
0
0
0
0
2
1
0
1
4
0
0
1
12
35
paging...
35
paging...
35
0
0
0
0
0
1
0
5
3
1
14
23
paging...
23
0
0
0
0
0
0
0
0
0
2
1
2
0
0
0
0
0
1
0
0
1
0
0
0
0
0
0
0
0
1
4
13
0
0
0
1
0
0
0
0
0
0
4
5
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [43]:
zerowaste_list

[{'address_name': '제주특별자치도 제주시 노형동 3058-1',
  'category_group_code': '',
  'category_group_name': '',
  'category_name': '가정,생활 > 슈퍼마켓 > 유기농슈퍼 > 한살림',
  'distance': '',
  'id': '972949833',
  'phone': '064-745-5988',
  'place_name': '한살림 제주담을매장',
  'place_url': 'http://place.map.kakao.com/972949833',
  'road_address_name': '제주특별자치도 제주시 월광로 12',
  'x': '126.463873888754',
  'y': '33.4826937068353'},
 {'address_name': '제주특별자치도 제주시 한림읍 상명리 1036',
  'category_group_code': '',
  'category_group_name': '',
  'category_name': '문화,예술 > 도서 > 서점',
  'distance': '',
  'id': '876962179',
  'phone': '010-8298-9884',
  'place_name': '책방소리소문 (휴업중)',
  'place_url': 'http://place.map.kakao.com/876962179',
  'road_address_name': '제주특별자치도 제주시 한림읍 중산간서로 4062',
  'x': '126.27166228801501',
  'y': '33.35801901156633'},
 {'address_name': '제주특별자치도 제주시 노형동 1021-2',
  'category_group_code': '',
  'category_group_name': '',
  'category_name': '사회,공공기관 > 단체,협회',
  'distance': '',
  'id': '322286925',
  'phone': '

In [44]:
zerowaste_list2 = []
# 최종 데이터가 담긴 리스트 중복값 제거
zerowaste_list = list(map(dict, collections.OrderedDict.fromkeys(tuple(sorted(d.items())) for d in zerowaste_list)))
for i in zerowaste_list:
    j = i['place_name']
    tel = i['phone']
    addr = i['address_name']
    url = i['place_url']
    x_list = float(i['y'])
    y_list = float(i['x'])
    zerowaste_list2.append({'store':j, 'tel':tel, 'address':addr, 'x':x_list, 'y':y_list, 'url' : url})
len(zerowaste_list2)

133

In [45]:
df = pd.DataFrame(zerowaste_list2, columns = ['store', 'x', 'y', 'tel', 'address','url'])
df

,store,x,y,tel,address,url
0,한살림 제주담을매장,33.482694,126.463874,064-745-5988,제주특별자치도 제주시 노형동 3058-1,http://place.map.kakao.com/972949833
1,책방소리소문 (휴업중),33.358019,126.271662,010-8298-9884,제주특별자치도 제주시 한림읍 상명리 1036,http://place.map.kakao.com/876962179
2,지구별가게,33.492685,126.475279,064-711-8291,제주특별자치도 제주시 노형동 1021-2,http://place.map.kakao.com/322286925
3,그림책방&카페노란우산,33.446052,126.430592,070-4158-9313,제주특별자치도 제주시 애월읍 광령리 2981-1,http://place.map.kakao.com/1827460043
4,북스페이스곰곰,33.479061,126.461211,010-5105-7433,제주특별자치도 제주시 도평동 48-10,http://place.map.kakao.com/1895166400
...,...,...,...,...,...,...
128,책빵자크르,35.533754,129.291175,052-268-2008,울산 남구 옥동 260-11,http://place.map.kakao.com/1379118545
129,베지경주제과 황리단길점,35.834579,129.209339,054-746-2270,경북 경주시 사정동 134,http://place.map.kakao.com/1212626765
130,지구맑음,35.540478,129.338563,,울산 남구 삼산동 1524-4,http://place.map.kakao.com/746166808
131,더벨루,35.967337,129.407161,070-8802-0320,경북 포항시 남구 오천읍 원리 936-7,http://place.map.kakao.com/439906510


In [36]:
x_list = list(map(lambda i:i['x'], zerowaste_list2))
y_list = list(map(lambda i:i['y'], zerowaste_list2))
tel_list = list(map(lambda i:i['tel'], zerowaste_list2))
zero_map = folium.Map(location=get_avg_point(x_list, y_list),zoom_start=7)
for x, y, t in zip(x_list, y_list, tel_list):
    marker = draw_marker(x, y, t)
    marker.add_to(zero_map)
zero_map